In [1]:
# Importamos las librerías necesarias
import pandas as pd
import folium
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
df = pd.read_csv('sistema-unico-de-atencion-ciudadana-2021.csv', delimiter=';', encoding='latin-1')
    
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df
    


C:\Users\Maty\AppData\Local\Temp\ipykernel_5128\1181956716.py:1: DtypeWarning: Columns (10,12,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('sistema-unico-de-atencion-ciudadana-2021.csv', delimiter=';', encoding='latin-1')


,contacto,periodo,categoria,subcategoria,prestacion,tipo_prestacion,fecha_ingreso,hora_ingreso,domicilio_comuna,domicilio_barrio,domiclio_calle,domicilio_altura,domicilio_esquina_proxima,lat,lon,canal,genero,estado_del_contacto,fecha_cierre_contacto
0,00173584/21,202104,TRÁMITES Y SERVICIOS,PROPUESTAS PARA LA MEJORA EN TRÁMITES,PROPUESTAS PARA LA MEJORA EN TRÁMITES,QUEJA,2021-04-13,11:54:30 a. m.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCS Web,Femenino,Abierto,NaN
1,00173599/21,202104,BARRIOS EMERGENTES,SERVICIO DE ELECTRICIDAD,CONSULTA DE ESTADO DE EXPEDIENTE ANTE LA AGENCIA DE PROTECCIÓN AMBIENTAL,SOLICITUD,2021-04-13,12:02:56 p. m.,COMUNA 10,Villa Luro,MILTON,456.0,NaN,"-34,63307579","-58,49888186",GCS Web,Femenino,Cerrado,2021-04-13
2,00173585/21,202104,LIMPIEZA Y RECOLECCIÓN,RESIDUOS VOLUMINOSOS,RETIRO DE ESCOMBROS / RESTOS DE OBRA,SERVICIO,2021-04-13,11:55:21 a. m.,COMUNA 2,Recoleta,RIOBAMBA,950.0,NaN,"-34,59795361","-58,39472064",GCS Web,Femenino,Cerrado,2021-04-15
3,00173621/21,202104,LIMPIEZA Y RECOLECCIÓN,RESIDUOS VOLUMINOSOS,RETIRO DE RESTOS DE JARDINERÍA DOMICILIARIA,SERVICIO,2021-04-13,12:14:07 p. m.,COMUNA 11,Villa Gral. Mitre,CONDARCO,1763.0,NaN,"-34,61246582","-58,47689153",App BA 147,Femenino,Cerrado,2021-04-15
4,00173634/21,202104,LIMPIEZA Y RECOLECCIÓN,RESIDUOS VOLUMINOSOS,RETIRO DE ESCOMBROS / RESTOS DE OBRA,SERVICIO,2021-04-13,12:22:34 p. m.,COMUNA 7,Flores,RIVERA INDARTE AV.,655.0,NaN,"-34,63681165","-58,45837221",App BA 147,Femenino,Cerrado,2021-04-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754633,00598882/21,202112,LIMPIEZA Y RECOLECCIÓN,RESIDUOS VOLUMINOSOS,Retiro de escombros / restos de obra,Servicio,2021-12-30,09:40:27 p. m.,COMUNA 9,Mataderos,NaN,7002.0,NaN,"-34,66234455","-58,50859528",GCS Web,Femenino,Abierto,NaN
754634,00598523/21,202112,LIMPIEZA Y RECOLECCIÓN,CESTOS Y CONTENEDORES,Vaciado de contenedor,Queja,2021-12-30,04:14:43 p. m.,COMUNA 1,San Nicolas,NaN,735.0,NaN,"-34,59987036","-58,38115543",GCS Web,Masculino,Cerrado,2022-01-02
754635,00597869/21,202112,LIMPIEZA Y RECOLECCIÓN,RESIDUOS VOLUMINOSOS,Retiro de escombros / restos de obra,Servicio,2021-12-30,10:03:10 a. m.,COMUNA 9,Parque Avellaneda,NaN,1470.0,NaN,"-34,65070682","-58,48779632",App BA 147,Masculino,Abierto,NaN
754636,00599078/21,202112,LIMPIEZA Y RECOLECCIÓN,LIMPIEZA DE VÍA PÚBLICA,Mejora de barrido,Queja,2021-12-31,08:13:36 a. m.,COMUNA 10,Floresta,NaN,175.0,NaN,"-34,63515329","-58,4780359",App BA 147,Masculino,Abierto,NaN


In [3]:
df['lat'] = df['lat'].astype(str).str.replace(',', '.').astype(float)
df['lon'] = df['lon'].astype(str).str.replace(',', '.').astype(float)

In [7]:
grid_size = 0.001  

lon_min, lon_max = df['lon'].min(), df['lon'].max()
lat_min, lat_max = df['lat'].min(), df['lat'].max()
lon_range = np.arange(lon_min, lon_max + grid_size, grid_size)
lat_range = np.arange(lat_min, lat_max + grid_size, grid_size)

heatmap_data = []

for lat in lat_range:
    for lon in lon_range:
        filtered_data = df[
            (df['lat'] >= lat) & (df['lat'] < lat + grid_size) &
            (df['lon'] >= lon) & (df['lon'] < lon + grid_size)
        ]
        
        # Calcular la puntuación promedio para esta grilla
        if len(filtered_data) > 0:
            avg_score = filtered_data.shape[0]  # Por ejemplo, cantidad de registros en la grilla

            heatmap_data.append({'lat': lat + grid_size / 2, 'lon': lon + grid_size / 2, 'score': avg_score})

# Crear el mapa de calor usando Plotly Express
fig = px.density_mapbox(heatmap_data, 
                        lat='lat', 
                        lon='lon', 
                        z='score',
                        radius=25, 
                        opacity=0.5, 
                        center=dict(lat=-34.6, lon=-58.5),  
                        zoom=10,  
                        mapbox_style='open-street-map')

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()